It is a pain to have to keep dumping compute and garbage text files just to regenerate J each time.
The point of this notebook is to carefully review the process of J calculation,
so that we can perform it just once and save to the result to a shared numpy zip file.

In [1]:
file_path = '/Users/lfinkbeiner/Documents/GitHub/HERA/notebooks/beam/'
beam_origin = 'HERA_4.9m_E-pattern_151MHz.txt'
beam_destination = 'ant1_s2'

In [2]:
%matplotlib notebook
import matplotlib.pyplot as plt
from RIMEz import beam_models
import numpy as np
import healpy as hp
from spin1_beam_model import cst_processing, jones_matrix_field
import rotations
import parser

ValueError: could not convert string to float: 

In [ ]:
processor = cst_processing.CSTDataProcessor(
    [file_path + beam_origin,
    file_path + beam_origin,
    file_path + beam_origin],
    np.array([150e6, 151e6, 152e6]),
    1, 1e-4
)

In [ ]:
processor.compute_spin1_harmonics()
processor.write_model_data(file_path, beam_destination)
spline_beam_func = beam_models.model_data_to_spline_beam_func(file_path + beam_destination + '.h5', np.array([150e6, 151e6, 152e6]))

In [ ]:
list_az = []
list_alt = []
for source in parser.obj_catalog:
    ra = np.radians(source.ra_angle)
    dec = np.radians(source.dec_angle)
    az, alt = rotations.eq_to_topo(ra, dec,
        np.radians(rotations.hera_lat),
        rotations.get_lst(), radians=True)
    list_az.append(az)
    list_alt.append(alt)

In [ ]:
"""
I am commenting this part out because we want to be able to
index into J as though we were indexing into obj_catalog.
list_az.sort()
list_alt.sort()
"""

# reformat into arrays so that the spline function can handle the single angles
list_az = np.array(list_az)
list_alt = np.array(list_alt)
print(list_az)
print(list_alt)

In [ ]:
J = spline_beam_func(150e6, list_alt, list_az)
J.shape

In [ ]:
J

In [ ]:
np.save("../J.npy", J, fix_imports=False)